In [2]:
!pip install llama-index llama_index.embeddings.huggingface llama-index-readers-file llama-index-llms-groq crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Aula 1

In [11]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

from google.colab import userdata
groq = userdata.get('GROQ_API_KEY')


llm = Groq(model="llama3-groq-70b-8192-tool-use-preview", api_key=groq)

Settings.llm = llm



In [12]:
import requests
from bs4 import BeautifulSoup

def buscar_mencoes_sem_api(termo: str) -> str:
    """
    Busca menções a um termo na web usando o Google Search (web scraping).

    Args:
        termo (str): O termo ou nome que você deseja buscar.

    Returns:
        str: Resultados encontrados (títulos e links) ou uma mensagem indicando que nada foi encontrado.
    """
    # URL de busca no Google
    url = f"https://www.google.com/search?q={termo.replace(' ', '+')}"

    # Cabeçalhos para simular um navegador (necessário para evitar bloqueios simples)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    # Fazendo a requisição
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        return "Não foi possível acessar o Google. Tente novamente mais tarde."

    # Parseando o HTML da página
    soup = BeautifulSoup(response.text, "html.parser")

    # Encontrando os resultados de busca
    resultados = soup.find_all("div", class_="tF2Cxc")  # Classe específica dos resultados do Google

    # Processando os resultados
    lista_resultados = []
    for resultado in resultados[:5]:  # Pegando apenas os 5 primeiros resultados
        titulo = resultado.find("h3").text if resultado.find("h3") else "Sem título"
        link = resultado.find("a")["href"] if resultado.find("a") else "Sem link"
        lista_resultados.append(f"Título: {titulo}\nLink: {link}")

    # Retornando os resultados formatados
    if lista_resultados:
        return "\n\n".join(lista_resultados)
    else:
        return "Nenhuma menção encontrada."

# Exemplo de uso
print(buscar_mencoes_sem_api("Carlos Llamando"))


Título: Calling Carlos: Llamando a Carlos - Books
Link: https://www.amazon.com/Calling-Carlos-Mar%C3%ADa-Victoria-Castillo/dp/1964792010

Título: Calling Carlos: Llamando a Carlos (Paperback)
Link: https://www.booksoup.com/book/9781964792019

Título: Calling Carlos: Llamando a Carlos (Paperback)
Link: https://www.avidbookshop.com/book/9781964792019

Título: Llamando a Carlos by Castillo, MarÍa Victoria, Brand New ...
Link: https://www.ebay.com/itm/196551782966

Título: Calling Carlos: Llamando a Carlos
Link: https://agendabookshop.com/products/calling-carlos-llamando-a-carlos?srsltid=AfmBOoqsbwFW2-tPfrPr-YIwM2eDbMtwjrxSAngrUhVgTrQfJkGi1v2b


In [13]:
from llama_index.core.tools import FunctionTool

In [14]:
ferramenta = FunctionTool.from_defaults(fn=buscar_mencoes_sem_api,name="Busca menções",description=(
    """    Busca menções a um termo na web usando o Google Search (web scraping).

    Args:
        termo (str): O termo ou nome que você deseja buscar.

    Returns:
        str: Resultados encontrados (títulos e links) ou uma mensagem indicando que nada foi encontrado."""
    ))

In [15]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [16]:
# Criando o agente
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta],
    verbose=True,
    allow_parallel_tool_calls=True
)

In [17]:
from llama_index.core.agent import AgentRunner

In [ ]:
agent = AgentRunner(agent_worker)

# Consultando o agente
response = agent.chat("Quais são as últimas menções sobre Carlos Llamando?")

# Aula 4

In [30]:
from transformers import pipeline
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import LlamaIndexTool


In [20]:
# Função para analisar o sentimento de um texto
def analisar_sentimento(texto: str) -> str:
    """
    Analisa o sentimento de um texto (positivo, negativo ou neutro).

    Args:
        texto (str): O texto a ser analisado.

    Returns:
        str: O sentimento detectado no texto.
    """
    sentiment_pipeline = pipeline("sentiment-analysis")
    resultado = sentiment_pipeline(texto)
    sentimento = resultado[0]["label"]
    score = resultado[0]["score"]
    return f"Sentimento: {sentimento}, Confiança: {score:.2f}"

In [28]:
# Criando ferramentas para o CrewAI
ferramenta2 = FunctionTool.from_defaults(
    fn=analisar_sentimento,
    name="analisar_sentimento",
    description="Analisa o sentimento de um texto e retorna se é positivo, negativo ou neutro."
)

In [31]:
tool_busca = LlamaIndexTool.from_tool(ferramenta)
tool_sentimento = LlamaIndexTool.from_tool(ferramenta2)

In [23]:
nvidia = userdata.get('nvidia-llama405b')

In [25]:
# Configurando o LLM
llm = LLM(
    model="nvidia_nim/meta/llama-3.3-70b-instruct",
    api_key=nvidia
)

In [33]:
# Criando o agente
agent = Agent(
    role="Agente de monitoramento",
    goal="Buscar menções sobre Carlos Llamando na web e analisar o sentimento associado.",
    backstory="Um agente especializado em monitoramento de reputação online para influenciadores digitais.",
    tools=[tool_busca, tool_sentimento],
    llm=llm
)

In [34]:
# Criando as tarefas
task1 = Task(
    description="Busque menções sobre Carlos Llamando na web.",
    expected_output="Lista de menções encontradas.",
    agent=agent
)

task2 = Task(
    description="Analise o sentimento das menções encontradas.",
    expected_output="Lista de sentimentos (positivo, negativo ou neutro) associados às menções.",
    agent=agent
)

In [35]:
# Criando o Crew
crew = Crew(
    agents=[agent],
    tasks=[task1, task2],
    verbose=1
)

# Executando o Crew
result = crew.kickoff()

# Exibindo os resultados
print("Resultados da busca e análise de sentimentos:")
print(result)

# Agent: Agente de monitoramento
## Task: Busque menções sobre Carlos Llamando na web.


# Agent: Agente de monitoramento
## Thought: Thought: Para buscar menções sobre Carlos Llamando na web, preciso utilizar a ferramenta de busca que tenho acesso, que é a "Busca menções". Com essa ferramenta, posso fornecer o termo que desejo buscar, que nesse caso é "Carlos Llamando".
## Using tool: Busca menções
## Tool Input: 
"{\"termo\": \"Carlos Llamando\"}"
## Tool Output: 
Título: Calling Carlos: Llamando a Carlos - Books
Link: https://www.amazon.com/Calling-Carlos-Mar%C3%ADa-Victoria-Castillo/dp/1964792010

Título: Calling Carlos: Llamando a Carlos (Paperback)
Link: https://www.booksoup.com/book/9781964792019

Título: Llamando a Carlos by Castillo, MarÍa Victoria, Brand New ...
Link: https://www.ebay.com/itm/196551782966

Título: Calling Carlos: Llamando a Carlos (Paperback)
Link: https://www.avidbookshop.com/book/9781964792019

Título: Calling Carlos: Llamando a Carlos
Link: https://agendab

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...



# Agent: Agente de monitoramento
## Thought: Thought: O sentimento da primeira menção é positivo.
## Using tool: analisar_sentimento
## Tool Input: 
"{\"texto\": \"Calling Carlos: Llamando a Carlos (Paperback)\"}"
## Tool Output: 
Sentimento: NEGATIVE, Confiança: 0.99


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.




# Agent: Agente de monitoramento
## Thought: Thought: O sentimento da segunda menção é negativo, o que é inesperado pois se tratava apenas de uma descrição de produto, mas vamos continuar com as demais menções.
## Using tool: analisar_sentimento
## Tool Input: 
"{\"texto\": \"Llamando a Carlos by Castillo, Mar\\u00cda Victoria, Brand New ...\"}"
## Tool Output: 
Sentimento: POSITIVE, Confiança: 1.00


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Busca menções
Tool Arguments: {'termo': {'description': None, 'type': 'str'}}
Tool Description:     Busca menções a um termo na web usando o Google Search (web scraping).

    Args:
        termo (str): O termo ou nome que você deseja buscar.

    Returns:
        str: Resultados encontrados (títulos e links) ou uma mensagem indicando que nada foi encontrado.
Tool Name: analisar_sentimento
Tool Arguments: {'texto': {'description': None, 'type': 'str'}}
Tool Description: Anal

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.




# Agent: Agente de monitoramento
## Thought: Thought: O sentimento da quarta menção é negativo, similar à segunda menção, o que pode indicar um padrão, mas é importante continuar.
## Using tool: analisar_sentimento
## Tool Input: 
"{\"texto\": \"Calling Carlos: Llamando a Carlos\"}"
## Tool Output: 
Sentimento: POSITIVE, Confiança: 0.98


# Agent: Agente de monitoramento
## Final Answer: 
1. Calling Carlos: Llamando a Carlos - Books - Sentimento: POSITIVO
2. Calling Carlos: Llamando a Carlos (Paperback) - Sentimento: NEGATIVO
3. Llamando a Carlos by Castillo, MarÍa Victoria, Brand New ... - Sentimento: POSITIVO
4. Calling Carlos: Llamando a Carlos (Paperback) - Sentimento: NEGATIVO
5. Calling Carlos: Llamando a Carlos - Sentimento: POSITIVO

Essa lista representa os sentimentos de cada menção encontrada na web relacionada a "Carlos Llamando".


Resultados da busca e análise de sentimentos:
1. Calling Carlos: Llamando a Carlos - Books - Sentimento: POSITIVO
2. Calling Carlos: Llamando